This notebook calculates the number of pixels in the iceplant polygons we have collected. 

This is, in a way, a maxium number of training data points for the iceplant class.

In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd

import sample_rasters as sr

In [2]:
aois = ['campus_lagoon','capitan','carpinteria','gaviota','point_conception']
years = [2020]        
n_pixels = []

for aoi in aois:
    for year in years:  
        # open polygons
        fp = sr.path_to_polygons(aoi, year)
        polys = gpd.read_file(fp)
        
        itemid =  polys.naip_id[0]
        item = sr.get_item_from_id(itemid)
        rast_reader = sr.get_raster_from_item(item)
        
        ice_polys = polys[polys.iceplant == 1]
        # convert to same crs as raster to properly calculate area of polygons
        if ice_polys.crs != rast_reader.crs:
            polys_match = ice_polys.to_crs(rast_reader.crs)
        else:
            polys_match = ice_polys

        # area of a single pixel from raster resolution    
        pixel_size = rast_reader.res[0]*rast_reader.res[1]

        # get approx number of pixels by dividing polygon area by pixel_size
        pixels_in_polys = polys_match.geometry.apply(lambda p: int((p.area/pixel_size))).to_numpy()
        n_pixels.append(sum(pixels_in_polys))
        
        


In [5]:
pd.DataFrame({'aoi':aois,'ice_pixels': n_pixels})

,aoi,ice_pixels
0,campus_lagoon,19368
1,capitan,8315
2,carpinteria,14929
3,gaviota,5387
4,point_conception,83419


In [7]:
sum(n_pixels)

131418